In [4]:
from project_functions_final_df import *
import pandas as pd
import geopandas as gpd
import numpy as np
import os 
from pathlib import Path  
import matplotlib.pyplot as plt

# Variable Assignments

In [5]:
clienttoken = "MLY|6271859919532158|486bcdece7a665eb89a5803f2994b4c9"
bounds = [(12.436781, 55.545432), (12.436781, 55.731815), (12.696966, 55.731815), (12.696966, 55.545432)]
my_shapefile_path = "koebenhavn.shp"
my_shp = gpd.read_file(my_shapefile_path)

In [6]:
ids_file = pd.read_csv('ids.csv')
IDs =  ids_file['image_id'].to_list()
np.random.seed(42)
IDs = list(np.random.choice(IDs, 100000))

In [9]:
strava_csv = pd.read_csv("data_from_laura/all_edges_monthly_jan_2022-dec_2022_ped/7f42d69dd7bd00b04081b9cbd0d05ab4e5f87353dcd7364c34b8e7c74d42109d-1676891223740.csv")
df_activity = StravaToDF(my_shp, bounds, strava_csv)

# Create dataset

### Get Detections for Mapillary

In [10]:
df_detections, dict_image = GetDetections_FaF(IDs, clienttoken, data_type = 'int')

Loading…: 100%|██████████████████| 100000/100000 [6:45:30<00:00,  4.11it/s]


### Export Detections Dataframe

In [11]:
current_path = os.getcwd()
filepath = Path(current_path+'/detections_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_detections.to_csv(filepath) 

### Create Model Data

In [12]:
pip = GetPointsInPoly(my_shp, clienttoken, bounds, IDs)
model_data = DFTransform(df_detections, pip, df_activity, dict_image, threshold = 0.005)

Loading…: 100%|██████████████████| 100000/100000 [3:13:46<00:00,  8.60it/s]
/home/s214584/project_functions_final_df.py:82: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_polygons["geometry"] = df_polygons["geometry"].buffer(buffer, cap_style = 2)


### Reassign Activity to Classes

In [13]:
ReassignActivity(model_data)

### Export Model Data

In [14]:
current_path = os.getcwd()
filepath = Path(current_path+'/modeldata_final1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
model_data.to_csv(filepath)